In [ ]:
# Reload functions every time
%load_ext autoreload 
%autoreload 2

In [ ]:
from pathlib import Path
from datetime import datetime
import pandas as pd
import sys


PROJECT_ROOT = Path().home() / 'HyMind'

# Patient flow package
USER_ROOT = Path().home()
sys.path.append(str(USER_ROOT / 'patientflow' / 'patientflow' ))

# Functions that sit outside the package
sys.path.append(str(USER_ROOT / 'patientflow' / 'functions' ))




In [ ]:
data_file_path = PROJECT_ROOT / 'dissemination' / 'data-raw'
data_file_path

model_file_path = PROJECT_ROOT / 'dissemination' / 'model-output' / 'trained-models'
model_file_path.mkdir(parents=True, exist_ok=True)



## Example of using this model for inference

In [ ]:
# load the admissions dataset set and pick a random date for inference
from ed_admissions_data_retrieval import ed_admissions_get_data

PATH_ED = 'HyMind/dissemination/data-raw/ED_visits.csv'
df = ed_admissions_get_data(PATH_ED)

# Load speciality dataset
path_spec_data = 'HyMind/dissemination/data-raw/specialty.csv'
df_spec = ed_admissions_get_data(path_spec_data)

prediction_time_ = (12, 0)
random_snapshot_datetime = df.loc[df.training_validation_test == 'test', 'snapshot_datetime'].sample(n=1, random_state=42).iloc[0]

episode_slices_df = df[(df.snapshot_datetime == random_snapshot_datetime) & (df.prediction_time == prediction_time_) & (df.current_ed_location != 'OTF')].copy()
slice_datetime = datetime.strptime(f'{random_snapshot_datetime} {12}:{0}', '%Y-%m-%d %H:%M')

In [ ]:
df.head()

#### Merge ED admissions data with specialty data

In our local implementation, the consults data are loaded with the other data at inference time. Therefore to replicate that here, we will merge the two datasets together

In [ ]:
# episode_slices_df
episode_slices_df = pd.merge( episode_slices_df, df_spec[['snapshot_id', 'consultation_sequence']],how = 'left')

In [ ]:
# Load the prediction window saved in the config file
import yaml

config_path = Path(PROJECT_ROOT / 'dissemination' )

with open(config_path / 'config.yaml', 'r') as file:
    config = yaml.safe_load(file)
    
prediction_window = str(int(config['prediction_window']/60))



In [ ]:
from ed_admissions_realtime_preds import create_predictions

specialties = ['medical', 'surgical', 'haem_onc', 'paediatric']
cdf_cut_points = [0.9, 0.7]

create_predictions(model_file_path, slice_datetime, episode_slices_df, specialties, prediction_window, cdf_cut_points) 
